# H5 file tree viewer

Workflow:<br/>
  1. Browse file  
  2. Keep an updated dictionaty with opened file objects and its properties - *h5_dict*
    * file_path
    * hf
    * root_attrs
    * root_properties
    * hf_root_items
    * hf_new_items
  3. Open/Load tree content - each node must have properties:
    * name - name of the nodes (group name or dataset name)
    * tp - type i.e file, group or dataset
    * obj_filepath - absolute full path of h5 file
    * obj_path - the path of the object (group or dataset) inside the h5 tree structure
    * obj_attributes - group or datasets attributes
    * obj_dtype - data type of datasets
    * obj_dshape - data shape of datasets
  4. Interactively (on_select) show its attributes

In [1]:
# imports and global variables
from IPython.display import Javascript, HTML, display
from ipywidgets import link, Layout, Tab, Box, HBox, VBox, IntSlider, FloatSlider, Text, Textarea, Label, Button, Image
from traitlets import Int, Float, Unicode, List, Tuple, Bool
from hdfviewer.widgets.PathSelector import PathSelector
from ipytree import Tree, Node
import h5py as h5

import time

h5_dict = {}
h5_objects = []

tree = Tree(multiple_selection=False)
tree_dict = {}
node_selected = None # current Node object selected

# Text widget to display selected_node props and attrs
name_text = Label(value = 'node selected')
file_text = Label(value = 'file selected')
dtype_text = Label()
dshape_text = Label()
attrs_text = Textarea(value = 'attributes displayed here',                      
                      description = '',
                      disabled = True,
                      continous_update = True,
                      layout = Layout(height = '50%', width = '100%')
                     )
tabs = Tab() # container with tabs

# h5nuvola logo
f_logo = open("/home/carlos/Documents/Elettra/h5nuvola/app_dev/h5nuvolaRemoteBrowser/static/logo_h5nuvola_updated.png", "rb")
image = f_logo.read()
h5nuvola_logo_item = Image(value=image,
                           format='png',
                           width=285,
                           height=60
                          )
h5nuvola_logo_item.layout.justify_content = 'center'

In [2]:
# define properties for each Node of the Tree
class H5Obj(Node):
    #name
    #icon
    tp = Unicode() # type - file, group or dataset
    obj_filepath = Unicode()
    obj_path = Unicode()
    obj_attributes = List()
    obj_children = Bool()
    obj_dtype = Unicode()
    obj_dshape = Tuple()

# routine to create Tree
def create_root_node(root_properties):
    global tree
    # create node
    root_node = H5Obj(name = root_properties['h5_name'],                      
                      icon = 'folder-o',
                      tp = root_properties['type'],
                      obj_filepath = root_properties['filepath'],
                      obj_path = '/',
                      obj_attributes = root_properties['attrs'],
                      obj_children = root_properties['children'],
                      obj_dtype = 'Group size',
                      obj_dshape = (root_properties['group_size'],) # tuple
                     )
    # add node to the tree
    tree.add_node(root_node)
    
    return root_node                      

# routine to expand a Node of the Tree
def expand_node(parent_node, new_items):
    for i in range(len(new_items)):
        # create new node
        new_node = H5Obj(name = new_items[i]['name'].split('/')[-1],
                         icon = new_items[i]['icon'],
                         tp = new_items[i]['type'], # 'file', 'group', 'dataset'
                         obj_filepath = parent_node.obj_filepath,
                         obj_path = new_items[i]['name'],                         
                         obj_children = new_items[i]['children'],
                         obj_dtype = new_items[i]['dtype'],
                         obj_dshape = new_items[i]['dshape'], # tuple
                         obj_attributes = new_items[i]['attrs']
                        )        
        parent_node.add_node(new_node)

    
# aux h5 methods
def h5_visit(name, obj):
    global h5_objects    
    h5_objects.append(obj)

def h5_get_items(items):
    items_list = [] # a list of dict containing all item's properties
    for item in items:
        name = item[1].name # str
        tp = '' # str
        icon = '' # str    
        children = None # boolean
        dtype = '' # str
        dshape = () # tuple
        attrs = [] # list
        
        if type(item[1]) == h5._hl.dataset.Dataset: # is dataset
            tp = 'dataset'
            icon = 'bars'
            children = False
            if h5.check_dtype(vlen=item[1].dtype) == bytes:
                dtype = 'string'
                dshape = (0,) # tuple
            else:
                dtype = str(item[1].dtype)
                dshape = item[1].shape # tuple           
        else: # is group
            tp = 'group'
            icon = 'folder-o'
            if len(item[1].items()) == 0:
                children = False
            else:
                children = True            
            dtype = 'Group size'
            dshape =  ( len(item[1].items()), ) # tuple
        
        # get attrs
        if len(item[1].attrs.keys()) == 0: # if there is no attributes
            pass
        else:
            for key in item[1].attrs.keys():
                attrs.append({'attr_name':key,
                              'attr_value':item[1].attrs[key].decode('utf-8')
                             })
        # set list        
        items_list.append({'name': name,
                           'type': tp,
                           'icon': icon,                           
                           'children': children,
                           'dtype': dtype,
                           'dshape': dshape,
                           'attrs': attrs
                          })
    return items_list

def set_attrs_text():
    global node_selected
    attrs_str = ''
    for attr in node_selected.obj_attributes:    
        attrs_str = attrs_str + attr['attr_name'] + ': ' + attr['attr_value'] + '\n\n'
    return attrs_str

In [16]:
# Server side file browser
#root_dir = '/run/user/1000/gvfs/sftp:host=tesla.elettra.eu,user=administrator/net/memory4twinmic/store/project/twinmic/scicomp/tesla-storage'
root_dir = '/home/carlos/h5files'
path = PathSelector(start_dir=root_dir, 
                    extensions=[".hdf", ".h5", ".nxs", ".ptyd", ".ptyr"])

# Button to load h5 file and add it to the Tree
button_load = Button(
    description = 'Load h5 file',
    disabled = False,
    button_style = '', # 'success', 'info', 'warning', 'danger' or ''
    tooltip = 'Click to load the selected h5 file',
    icon = ''
)
# Button to close h5 file and remove it from the Tree
button_close = Button(
    description = 'Close h5 file',
    disabled = False,
    button_style = '',
    tooltip = 'Click to close the selected h5 file',
    icon = ''
)


def on_button_load_clicked(btn_obj):
    global h5_dict, hf_objects, h5_new_items
    if path.file:
        if path.file not in h5_dict:
            print("Loading %s"%(path.file))
            hf = h5.File(path.file, 'r')
            h5_dict[path.file] = {'hf': hf}
            h5_name = str(hf.filename).split('/')[-1]
            h5_dict[path.file]['h5_name'] = h5_name
            h5_objects = []
            hf.visititems(h5_visit)
            h5_dict[path.file]['h5_objects'] = h5_objects
            
            # check if it has children
            if len(hf.items()) == 0:
                children = False
            else:
                children = True
            
            root_attrs = []
            if len(hf.attrs.keys()) == 0:
                pass
            else:
                for key in hf.attrs.keys():
                    root_attrs.append([ key, hf.attrs[key].decode('utf-8') ])           
            root_properties = {'h5_name': h5_name,
                               'type': 'file',
                               'filepath': path.file,
                               'attrs': root_attrs,
                               'children': children,
                               'group_size': len(hf.items())
                              }
            h5_dict[path.file]['root_properties'] = root_properties
            
            # create Tree and retrieve root node created
            tree_dict[h5_name] = create_root_node(root_properties)
            # get new items
            h5_new_items = h5_get_items(hf.items())
            
            if root_properties['children']:
                expand_node(tree_dict[h5_name], h5_new_items)
            
            tabs.selected_index = 1
                    
        else:
            print("File %s already opened"%(path.file))
    else:
        print("Please select a file on the File Browser widget")

def on_button_close_clicked(btn_obj):
    global h5_dict
    print('Close h5 file')
    # close h5 file object and delete from h5_dict
    h5_dict[node_selected.obj_filepath]['hf'].close()
    del h5_dict[node_selected.obj_filepath]
    # remove from tree
    tree.remove_node(node_selected)    
 
button_load.on_click(on_button_load_clicked)
button_close.on_click(on_button_close_clicked)

# define and bind tree event handler
def on_selected_change(change):
    global node_selected, name_text, button_close, attrs_text
    node_selected = change['new'][0]
    if node_selected.obj_children and node_selected.nodes == (): # if node has children and have no nodes expand it
        h5_new_items = h5_get_items( h5_dict[node_selected.obj_filepath]['hf'][node_selected.obj_path].items() )
        expand_node(node_selected, h5_new_items)
    if node_selected.tp != 'file':
        button_close.disabled = True
    else:
        button_close.disabled = False
    # update name_text, file_text
    name_text.value = node_selected.obj_path
    file_text.value = node_selected.obj_filepath
    dtype_text.value = node_selected.obj_dtype
    dshape_text.value = str(node_selected.obj_dshape)
    # show attributes
    attrs_text.value = set_attrs_text()
    
    
tree.observe(on_selected_change, names='selected_nodes')

# Display widgets
# VBox([Label('Browser'), 
#       path.widget, 
#       button_load, 
#       HBox([ VBox([Label('TreeViewer'), tree]), VBox([Label('Selection'), Label('   '), Label('File'), HBox([file_text, button_close]), Label('Object'), name_text]) ]) ])

# Layout and Display widgets - Using Flexbox
tree.layout = Layout(width = '100%',
                     height = '800px',
                     overflow_x = 'scroll',
                     overflow_y = 'scroll')
# items for Flexbox
file_browser_item = VBox([ Label('Browser'), path.widget, button_load, button_close ])
tree_viewer_item = VBox([ Label('H5 Content'), tree ])
attrs_item = VBox([Label('Selection'),
                   Label('   '),
                   Label('File'),
                   file_text,
                   Label('Object'),
                   name_text,
                   dtype_text,
                   dshape_text,
                   Label('Attributes'),
                   attrs_text
                  ])

box_layout = Layout(display = 'flex', # 'inline-flex', 'flex'
                    flex_flow = 'column nowrap', # 'column-reverse', 'column', 'row', 'row-reverse' | 'nowrap', 'wrap', 'wrap-reverse'
                    justify_content = 'space-around', # 'flex-start', 'flex-end', 'center', 'space-between', 'space-around' 
                    align_items = 'stretch', # 'flex-start', 'flex-end', 'center', 'baseline', 'stretch'
                    border = 'solid 10px rgb(0,130,196)',
                    width = '100%'
                   )

hbox_layout = Layout(display = 'inline-flex',
                     flex = '1 1 auto',
                     flex_flow = 'row nowrap',
                     justify_content = 'space-around',
                     align_items = 'stretch',
                     border = 'solid 1px black',
                     width = '100%'
                    )

# children for Tabs
children_tabs = [ file_browser_item, HBox([ tree_viewer_item, attrs_item ], layout = hbox_layout) ]

# Trying Tabs
tabs.children = children_tabs
tabs.set_title(0, 'File Browser')
tabs.set_title(1, 'H5 Content')

#children = VBox([ h5nuvola_logo_item, HBox([file_browser_item, tree_viewer_item, attrs_plot_item], layout = hbox_layout) ])
children = VBox([ h5nuvola_logo_item, tabs ])
box = Box(children = [children],
          layout = box_layout )



In [17]:
box

Box(children=(VBox(children=(Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x01\x1d\x00\x00\x00<\x0…

In [37]:
# remove and add tabs in Tab() widget
t = tabs.children
t = list(t)
del t[1]
# t = tuple(t)
tabs.children  = t

In [38]:
tabs.children = children_tabs